In [1]:
import requests as r
import pandas as pd
from datetime import datetime 
import shutil

In [2]:
data = pd.read_pickle('./data.pkl')

data['known_langs'] = data.repos_info.map(lambda l: {x['language'] for x in l} - {None})
data['starred_langs'] = data.starred_info.map(lambda l: {x['language'] for x in l} - {None})

data['all_langs'] = data[['known_langs', 'starred_langs']].apply(lambda x: x['known_langs'].union(x['starred_langs']), axis=1)
data['all_langs'] -= {'JavaScript', 'CSS', 'HTML'}
data['all_langs'] = data.all_langs.transform(lambda s: s.union({'JavaScript/HTML'}))

In [3]:
langs_data = data.all_langs

In [4]:
matrix_intesection = pd.DataFrame(index=langs_data.index)
matrix_union = pd.DataFrame(index=langs_data.index)

for x in langs_data.index:
    langs = langs_data[x]
    matrix_intesection[x] = langs_data.map(lambda x: x.intersection(langs))
    matrix_union[x] = langs_data.map(lambda x: x.union(langs))

In [5]:
jaccard = matrix_intesection.applymap(len) / matrix_union.applymap(len)

In [53]:
links = []
for i in range(len(jaccard)):
    for j in range(i+1, len(jaccard)):
        ind = jaccard.index[i]
        col = jaccard.index[j]
        links.append({'source': ind, 'target': col, 'value': float(jaccard.loc[ind][col]), 'key': ind + col if ind <= col else col + ind})

In [7]:
data['age'] = (datetime.today() - pd.to_datetime(data.created_at)).dt.days

In [8]:
def get_image(url, name):
    resp = r.get(url, stream=True)
    resp.raw.decode_content = True
    ext = resp.headers['Content-Type'].split('/')[-1]
    name = name + '.' + ('png' if ext == 'gif' else ext)
    shutil.copyfileobj(resp.raw, open(name, 'wb'))
    return name[7:]

In [9]:
data['image_url'] = data[['avatar_url']].reset_index().apply(lambda x: get_image(x[1], '../img/' + x[0]), axis=1).tolist()

In [96]:
def similars(s):
    return list(s.sort_values().drop(s.name).iloc[-2:].index)

In [97]:
data['most_similar'] = jaccard.apply(similars, axis=1)

In [98]:
nodes = data[['name', 'all_langs', 'age', 'image_url', 'most_similar']].reset_index().rename(columns={'index': 'id'})

In [99]:
nodes['all_langs'] = nodes['all_langs'].map(list)

In [100]:
nodes['group'] = nodes['all_langs'].map(len)

In [101]:
nodes = nodes.to_dict(orient='records')

In [102]:
final_data = {'nodes': nodes, 'links': links}

In [103]:
from json import dump
dump(final_data, open('../data/data.json', 'w'))